Imports

In [14]:
from demucs import pretrained
from transformers import AutoTokenizer, ClapModel

from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from src.models.stem_separation.ATHTDemucs_v2 import AudioTextHTDemucs

from utils import load_config
from src.train import load_checkpoint

Initialize our `AudioTextHTDemucs` Model


In [9]:
# Load pretrained models
print("Loading pretrained HTDemucs...")
htdemucs = pretrained.get_model('htdemucs').models[0]

print("Loading CLAP model...")
clap = ClapModel.from_pretrained("laion/clap-htsat-unfused")
tokenizer = AutoTokenizer.from_pretrained("laion/clap-htsat-unfused")

# Initialize model components
cfg = load_config("config.yaml")
model_dim = cfg["model"]["model_dim"]
text_dim = cfg["model"]["text_dim"]
n_heads = cfg["model"]["num_heads"]


model = AudioTextHTDemucs(
    htdemucs_model=htdemucs,
    clap_encoder=clap,
    clap_tokenizer=tokenizer,
    model_dim=model_dim,
    text_dim=text_dim,
    num_heads=n_heads,
)
print(f"Model Summary:\n{model}")

Loading pretrained HTDemucs...
Loading CLAP model...
Model Summary:
AudioTextHTDemucs(
  (htdemucs): HTDemucs(
    (encoder): ModuleList(
      (0): HEncLayer(
        (conv): Conv2d(4, 48, kernel_size=(8, 1), stride=(4, 1), padding=(2, 0))
        (norm1): Identity()
        (rewrite): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1))
        (norm2): Identity()
        (dconv): DConv(
          (layers): ModuleList(
            (0): Sequential(
              (0): Conv1d(48, 6, kernel_size=(3,), stride=(1,), padding=(1,))
              (1): GroupNorm(1, 6, eps=1e-05, affine=True)
              (2): GELU(approximate='none')
              (3): Conv1d(6, 96, kernel_size=(1,), stride=(1,))
              (4): GroupNorm(1, 96, eps=1e-05, affine=True)
              (5): GLU(dim=1)
              (6): LayerScale()
            )
            (1): Sequential(
              (0): Conv1d(48, 6, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
              (1): GroupNorm(1, 6, eps=1e-05, 

Load in sample weights

In [19]:
# Load in the model weights from a checkpoint
lr = float(cfg["training"]["optimizer"]["lr"])
optimizer = AdamW(
    model.parameters(),
    lr=lr,
    weight_decay=float(cfg["training"]["optimizer"]["weight_decay"]),
    betas=(0.9, 0.999),
)

scheduler = CosineAnnealingLR(
    optimizer,
    T_max=cfg["training"]["num_epochs"],
    eta_min=lr * 0.01,
)

model2 = AudioTextHTDemucs(
    htdemucs_model=htdemucs,
    clap_encoder=clap,
    clap_tokenizer=tokenizer,
    model_dim=model_dim,
    text_dim=text_dim,
    num_heads=n_heads,
)
load_checkpoint(model2, optimizer, scheduler, "checkpoints/2025_11_30_batch4/best_model.pt")
model2

Loaded checkpoint from epoch 10


AudioTextHTDemucs(
  (htdemucs): HTDemucs(
    (encoder): ModuleList(
      (0): HEncLayer(
        (conv): Conv2d(4, 48, kernel_size=(8, 1), stride=(4, 1), padding=(2, 0))
        (norm1): Identity()
        (rewrite): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1))
        (norm2): Identity()
        (dconv): DConv(
          (layers): ModuleList(
            (0): Sequential(
              (0): Conv1d(48, 6, kernel_size=(3,), stride=(1,), padding=(1,))
              (1): GroupNorm(1, 6, eps=1e-05, affine=True)
              (2): GELU(approximate='none')
              (3): Conv1d(6, 96, kernel_size=(1,), stride=(1,))
              (4): GroupNorm(1, 96, eps=1e-05, affine=True)
              (5): GLU(dim=1)
              (6): LayerScale()
            )
            (1): Sequential(
              (0): Conv1d(48, 6, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
              (1): GroupNorm(1, 6, eps=1e-05, affine=True)
              (2): GELU(approximate='none')
           

Dimension Analysis

In [ ]:
from torchinfo import summary

# HTDemucs forward pass
summary(model2.htdemucs, input_size=(1, 2, 44100 * 6))

Layer (type:depth-idx)                             Output Shape              Param #
HTDemucs                                           [1, 4, 2, 264600]         --
├─ModuleList: 1-8                                  --                        (recursive)
│    └─HEncLayer: 2-1                              [1, 48, 85995]            --
│    │    └─Conv1d: 3-1                            [1, 48, 85995]            (816)
│    │    └─Identity: 3-2                          [1, 48, 85995]            --
│    │    └─DConv: 3-3                             [1, 48, 85995]            (3,588)
│    │    └─Conv1d: 3-4                            [1, 96, 85995]            (4,704)
│    │    └─Identity: 3-5                          [1, 96, 85995]            --
├─ModuleList: 1-9                                  --                        (recursive)
│    └─HEncLayer: 2-2                              [1, 48, 512, 336]         --
│    │    └─Conv2d: 3-6                            [1, 48, 512, 336]         (1,584)

In [35]:
# CLAP forward pass
device = cfg["model"]["device"]
clap_input = tokenizer(
    ["Extract drums"],
    padding=True,
    return_tensors="pt"
)
clap_input = {k: v.to(device) for k, v in clap_input.items()}

summary(model2.clap.text_model, input_data={**clap_input})

Layer (type:depth-idx)                                  Output Shape              Param #
ClapTextModel                                           [1, 768]                  --
├─ClapTextEmbeddings: 1-1                               [1, 5, 768]               --
│    └─Embedding: 2-1                                   [1, 5, 768]               (38,603,520)
│    └─Embedding: 2-2                                   [1, 5, 768]               (768)
│    └─Embedding: 2-3                                   [1, 5, 768]               (394,752)
│    └─LayerNorm: 2-4                                   [1, 5, 768]               (1,536)
│    └─Dropout: 2-5                                     [1, 5, 768]               --
├─ClapTextEncoder: 1-2                                  [1, 5, 768]               --
│    └─ModuleList: 2-6                                  --                        --
│    │    └─ClapTextLayer: 3-1                          [1, 5, 768]               (7,087,872)
│    │    └─ClapTextLayer: